0 1 1 0 0 0  
1 0 1 1 0 0  
1 1 0 0 0 0  
0 1 0 0 1 1
0 0 0 1 0 1
0 0 0 1 1 0



In [1]:
import numpy as np
from typing import List, Tuple
from collections import defaultdict, deque
import itertools
from functools import reduce
from itertools import product

# Graph definition

class Graph:
    def __init__(self, adjMatrix: np.ndarray):
        self.adjMatrix = adjMatrix
        self.n = len(adjMatrix)
        self.vertexSet = list(range(self.n))
        self.edgeSet = self.buildEdgeSet()
        self.degreeSet = self.buildDegreeSet()
        self.edgePairs = {(u, v): (v, u) for (u, v) in self.edgeSet}

    def buildEdgeSet(self):
        edgeSet = set()
        for i in range(self.n):
            for j in range(0, self.n):
                if self.adjMatrix[i][j] != 0:
                    edgeSet.add((i, j))
        return edgeSet

    def buildDegreeSet(self):
        degreeSet = []
        for i in range(self.n):
            degree = sum(1 for j in range(self.n) if self.adjMatrix[i][j] != 0)
            degreeSet.append((i, degree))
        return degreeSet
        
    def neighbors(self, vertex: int):
        for j in range(self.n):
            if self.adjMatrix[vertex][j] != 0 or self.adjMatrix[j][vertex] != 0:
                yield j

    def inNeighbors(self, vertex: int):
        for j in range(self.n):
            if self.adjMatrix[j][vertex] != 0:
                yield j

    def outNeighbors(self, vertex: int):
        for j in range(self.n):
            if self.adjMatrix[vertex][j] != 0  :
                yield j


    


# Function definition
def buildSpanningTree(graph: Graph):
    visited = [False] * graph.n
    spanningTreeEdges = set()

    # Start with vertex of smallest degree (break ties by ID)
    start_vertex = min(graph.degreeSet, key=lambda x: (x[1], x[0]))[0]
    visited[start_vertex] = True

    while len(spanningTreeEdges) < graph.n - 1:
        candidates = []

        for (u, v) in graph.edgeSet:
            if visited[u] and not visited[v]:
                visited_part = u
                unvisited_part = v
            elif visited[v] and not visited[u]:
                visited_part = v
                unvisited_part = u
            else:
                continue  # skip if both visited or both unvisited

            deg_visited = dict(graph.degreeSet)[visited_part]
            deg_unvisited = dict(graph.degreeSet)[unvisited_part]

            # For sorting: (unvisited deg, visited deg, unvisited ID, visited ID)
            candidates.append((deg_unvisited, deg_visited, unvisited_part, visited_part))

        if not candidates:
            raise ValueError("Graph is not connected; cannot build spanning tree")

        # Sort candidates based on the given criteria
        candidates.sort()
        _, _, u, v = candidates[0]

        # Ensure edge is stored as (smaller, larger)
        edge = (min(u, v), max(u, v))
        spanningTreeEdges.add(edge)

        visited[u] = True
        visited[v] = True  # only one of them was unvisited, so it's safe

    return spanningTreeEdges



def findChordAndCycles(adjMatrix, edgeSet, spanningTree):
    """
    Given an adjacency matrix, full edge set, and spanning tree,
    returns the chords and fundamental cycles.

    Parameters:
        adjMatrix (np.ndarray): The adjacency matrix of the graph.
        edgeSet (set of (int, int)): All edges in the graph, with u < v enforced.
        spanningTree (set of (int, int)): Edges forming a spanning tree, with u < v enforced.

    Returns:
        chords (set of (int, int)): Edges not in the spanning tree.
        fundamentalCycles (list of tuple of (int, int)): List of cycles formed by chords.
    """
    fullEdgeSet = set((min(u, v), max(u, v)) for (u, v) in edgeSet)
    treeEdges = set((min(u, v), max(u, v)) for (u, v) in spanningTree)

    chords = fullEdgeSet - treeEdges

    treeAdj = defaultdict(list)
    for u, v in treeEdges:
        treeAdj[u].append(v)
        treeAdj[v].append(u)

    def findPath(start, end):
        parent = {start: None}
        queue = deque([start])
        while queue:
            node = queue.popleft()
            if node == end:
                break
            for neighbor in treeAdj[node]:
                if neighbor not in parent:
                    parent[neighbor] = node
                    queue.append(neighbor)

        path = []
        current = end
        while parent[current] is not None:
            prev = parent[current]
            path.append((min(current, prev), max(current, prev)))
            current = prev
        path.reverse()
        return path

    fundamentalCycles = []
    for u, v in chords:
        cyclePath = findPath(u, v)
        cyclePath.append((min(u, v), max(u, v)))  # the chord
        fundamentalCycles.append(set(cyclePath))

    return chords, fundamentalCycles








def userInput():
    n = int(input("Enter the size of the transition matrix (n x n): "))
   
    print(f"Enter the transition matrix row by row (space-separated entries for each row of size {n}):")

    matrix = []
    

    for i in range(n):
        while True:
            try:
                row = list(map(float, input(f"Row {i+1}: ").strip().split()))
                if len(row) != n:
                    raise ValueError("Incorrect number of entries.")
                matrix.append(row)
                
                break
            except ValueError as e:
                print(f"Invalid input: {e}. Please enter exactly {n} numbers.")

    return matrix





def xor_sets(sets_of_edges):
    """Perform symmetric difference (XOR) over a list of sets."""
    return reduce(set.symmetric_difference, sets_of_edges)

def findEvenDegreeSubGraph(n, edgeSet, fundamentalCycles):
    """
    Find all disjoint cycles by XOR-ing combinations of fundamental cycles.
    
    Arguments:
    - n: maximum number of cycles to combine
    - edgeSet: unused in current logic but could be used to restrict results
    - fundamentalCycles: list of sets of edges, each representing a fundamental cycle
    
    Returns:
    - List of sets representing disjoint cycles (non-empty XOR results)
    """
    evenDegreeSubGraph = []
    maxComLen = min(n, len(fundamentalCycles))

    for r in range(1, maxComLen + 1):  # from 1 up to maxComLen
        for combo in itertools.combinations(fundamentalCycles, r):
            xor_result = xor_sets(combo)
            if xor_result:  # only keep non-empty cycles
                evenDegreeSubGraph.append(xor_result)

    return evenDegreeSubGraph


    return evenSubgraphs



def disjointCycles(evenDegreeSubGraphs, chords):
    """
    Filters even-degree subgraphs to find true disjoint cycles (degree 2 at all vertices).
    
    Parameters:
        evenDegreeSubGraphs: list of sets of edges (each a candidate subgraph)
        chords: set of chord edges (used to report which chords are present in the cycle)
    
    Returns:
        List of valid disjoint cycles (each is a set of edges)
    """
    disjointCyclesList = []
    dCEdgeSet= []
    for i, edgeSet in enumerate(evenDegreeSubGraphs):
        # Count vertex degrees
        degree = defaultdict(int)
        for u, v in edgeSet:
            degree[u] += 1
            degree[v] += 1

        # Check that all vertices have degree 2
        if all(deg == 2 for deg in degree.values()):
            # Find which chords are in this cycle
            cycleChords= set(edge for edge in edgeSet if edge in chords)

            disjointCyclesList.append((cycleChords, edgeSet))

            # Print result
            chordStr = " ".join(str(ch) for ch in sorted(cycleChords))
            edgeStr = sorted(edgeSet)
            print(f"Disjoint cycle {len(disjointCyclesList)} chords {chordStr}: {edgeStr}")

    return disjointCyclesList

def verticesFromEdges(edges):
    seen = set()
    for u, v in edges:
        if u not in seen:
            seen.add(u)
            yield u
        if v not in seen:
            seen.add(v)
            yield v



def buildGraph(edgeSet, vertexSet,n):
    

    adjMatrix = np.zeros((n, n), dtype=float)

    for u, v in edgeSet:
        if u in vertexSet and v in vertexSet:
            adjMatrix[u][v] = 1

    return Graph(adjMatrix)



        

    


def listOrientedCycles(graph, edgeSet, chords):
    
    foundCycles = []
    analyzed = []
    vertexSet = list(verticesFromEdges(edgeSet)) 
    subGraph = buildGraph(edgeSet, vertexSet,graph.n)   
    print (f"vertex set of sub graph:{subGraph.vertexSet}")
    print (f"edge set of sub graph:{subGraph.edgeSet}")
    # for all graph. we should have subGraph.           added
    while len(analyzed) < len(vertexSet):
        newCycle = []

        # Find a starting vertex for the loop
        for i in vertexSet:
            if i not in analyzed:
                startVertex = i
                break

        currentVertex = startVertex
        #currentNeighbors = subGraph.neighbors(currentVertex)
        newCycle.append(startVertex)
        analyzed.append(startVertex)
        while True:
            #print(f"currentNeighbors is :{list(currentNeighbors)} ")
            #print(f"currentvortex is :{currentVertex} ")
            #print(f"analyzed is :{analyzed} ")
            for j in subGraph.neighbors(currentVertex):
                if j not in analyzed :
                    
                    newCycle.append(j)                    
                    currentVertex = j
                    analyzed.append(j)
                    #print(f"analyzed is :{analyzed} ")
                    break
                    
            currentNeighbors = subGraph.neighbors(currentVertex)   
            
            if all ( elem in analyzed for elem in   subGraph.neighbors(currentVertex)):
                break

        foundCycles.append(newCycle)

    return foundCycles

def iso_vertecies (graph, vortexSet):
    n = graph.n
    
    subAdjMatrix = np.copy(graph.adjMatrix)
    for i in vortexSet:
        for j in range(n):
            subAdjMatrix[i][j]=0
            subAdjMatrix[j][i]=0
    subGraph= Graph(subAdjMatrix)
    subGraph.vertexSet = [v for v in graph.vertexSet if v not in vortexSet]
    return subGraph




def all_matchings(G):
    if len(G.vertexSet) == 0:
        yield []
    elif len(G.vertexSet) == 1:
        yield [G.vertexSet[0]]
    else:
        v = G.vertexSet[0]

        #removing a vertex 
        for submatching in all_matchings(iso_vertecies(G, [v])):
            yield submatching + [v]

        #removing an edge which has starting vertex v
        for w in G.outNeighbors(v):
            newG = iso_vertecies(G, [v, w])
            for submatching in all_matchings(newG):
                yield submatching + [(v, w)]

        #removing an edge which has finishing vertex v
        #for w in G.inNeighbors(v):
        #    newG = iso_vertecies(G, [v, w])
        #    for submatching in all_matchings(newG):
        #        yield submatching + [(w, v)]

        #NOTE: the pairs (u,v) and (v,u) were counted twice. therefore the inNeighbors we commented. maybe this has to be changed   
        

def edge_in_cycle_orientation(edge, cycle):
    """Check orientation of edge in cycle."""
    u, v = edge
    for i in range(len(cycle)):
        if cycle[i] == u and cycle[(i + 1) % len(cycle)] == v:
            return 1  # same orientation
        if cycle[i] == v and cycle[(i + 1) % len(cycle)] == u:
            return -1  # reversed
    return 0  # not found

def generate_all_oriented_cycles_and_signed_chords(cycles, chords):
    all_orientations = list(product([False, True], repeat=len(cycles)))  # True = reverse
    results = []

    for orientation in all_orientations:
        oriented_cycles = []
        signed_chords = []

        # Apply orientation to cycles
        for reverse, cycle in zip(orientation, cycles):
            oriented_cycles.append(list(reversed(cycle)) if reverse else cycle[:])

        # Determine sign of chords
        for u, v in chords:
            sign = None
            for cycle in oriented_cycles:
                if u in cycle and v in cycle:
                    direction = edge_in_cycle_orientation((u, v), cycle)
                    if direction == 1:
                        sign = (u, v)
                    elif direction == -1:
                        sign = f"-({u}, {v})"
                    break
            if isinstance(sign, str):
                signed_chords.append(sign)
            else:
                signed_chords.append(sign)

        # Collect result
        results.append((oriented_cycles, signed_chords))

    return results

import itertools
import copy
import math

import itertools

import itertools
import math

def filter_pairPoints(pairPoints, vertices_to_remove_copy):
    filtered = []
    for sublist in pairPoints:
        # Remove the whole sublist if it contains a tuple with any vertex to remove
        if any(isinstance(item, tuple) and any(v in vertices_to_remove_copy for v in item) for item in sublist):
            continue
        # Otherwise, remove only individual vertices
        new_sublist = [item for item in sublist if not (isinstance(item, int) and item in vertices_to_remove_copy)]
        if new_sublist:
            filtered.append(new_sublist)
    return filtered

def process_pairPoints(pairPoints_copy):
    expressions = []
    for group in pairPoints_copy:
        bracket_count = sum(1 for item in group if isinstance(item, tuple))
        sign = "-" if bracket_count % 2 == 1 else ""
        terms = []

        # First add all integers
        for item in group:
            if isinstance(item, int):
                terms.append(f"w{item}")
        # Then add all tuples as wij*wji
        for item in group:
            if isinstance(item, tuple):
                i, j = item
                terms.append(f"w{i}{j}*w{j}{i}")

        expr = f"{sign}{'*'.join(terms)}"
        expressions.append(f"({expr})")
    return "+".join(expressions)

def process_orientedCycles(orientedCycles):
    sign = 1
    sqrt_terms = []
    for cycle in orientedCycles:
        length = len(cycle)
        if length % 2 == 0:
            sign *= -1  # even length → flip sign
        else:
            sign *= 1   # odd length → keep sign

        # Collect forward and reverse products
        for i in range(length):
            i1 = cycle[i]
            i2 = cycle[(i + 1) % length]
            sqrt_terms.append(f"w{i1}{i2}")
        for i in range(length - 1, -1, -1):
            i1 = cycle[i]
            i2 = cycle[i - 1 if i > 0 else -1]
            sqrt_terms.append(f"w{i1}{i2}")

    sign_str = "+1" if sign == 1 else "-1"
    sqrt_expr = f"{sign_str}*2*sqrt(" + "*".join(sqrt_terms) + ")"
    return sqrt_expr

def CumulantGenerator(pairPoints, vertices_to_remove, orientedCycles):
    result = []

    for i in range(len(vertices_to_remove) + 1):
        combos = list(itertools.combinations(vertices_to_remove, i))
        gs_brackets = []

        for combo in combos:
            pairPoints_copy = filter_pairPoints(pairPoints, combo)
            gs_inner = process_pairPoints(pairPoints_copy)
            gs_brackets.append(f"[{gs_inner}]")

        # Combine all bracket expressions for this i
        GS_expression = "+".join(gs_brackets)
        sqrt_term = process_orientedCycles(orientedCycles)
        GS_full = f"{GS_expression}*{sqrt_term}"
        result.append(GS_full)

    return result










# Run
graph = Graph(userInput())
print (f"{graph.edgeSet}")
print(type(graph.edgeSet))
spanningtree = buildSpanningTree(graph)
chords, fundamentalCycles = findChordAndCycles(graph.adjMatrix, graph.edgeSet, spanningtree)

print(f"\nChords: {chords}\n")
for i, cycle in enumerate(fundamentalCycles):
    print(f"Fundamental cycle {i+1}: {cycle}")

# Find even-degree subgraphs
evenDegreeSubGraphs = findEvenDegreeSubGraph(graph.n, graph.edgeSet, fundamentalCycles)

# Filter disjoint cycles from even-degree subgraphs and print them
disjointCyclesList = disjointCycles(evenDegreeSubGraphs, chords)
print(f"disjoint cycle list {disjointCyclesList}")




 # Print all even-degree subgraphs (unfiltered)
print("\nAll Even Degree Subgraphs:")
for j, subgraph in enumerate(evenDegreeSubGraphs):
    print(f"  Even degree subgraph {j+1}: {sorted(subgraph)}")

#print neighbors
for i in graph.vertexSet:  # You can iterate directly over the set
    print(f"neighbors of {i} = {list(graph.neighbors(i))}")
#    print(f"in neighbors of {i} = {list(graph.inNeighbors(i))}")
#    print(f"out neighbors of {i} = {list(graph.outNeighbors(i

# Run disjoint cycle detection
disjointCyclesList = disjointCycles(evenDegreeSubGraphs, chords)

print (disjointCyclesList)




print("\n--- Processing Oriented Cycles from Disjoint Cycles ---")

for idx, (chordSet, edgeSet) in enumerate(disjointCyclesList):
    print(f"\n### Disjoint Cycle {idx + 1} ###")
    print(f"Chords: {sorted(chordSet)}")
    print(f"Edges: {sorted(edgeSet)}")

    orientedCycles = listOrientedCycles(graph, edgeSet, chords)
    print(f"oriented cycles {orientedCycles}")

    # Enumerate all orientations
    print("\nEnumerating all oriented variants:")
    results = generate_all_oriented_cycles_and_signed_chords(orientedCycles, sorted(chordSet))

    for oriented_cycles, signed_chords in results:
        print(f"oriented cycles: {oriented_cycles}")
        print(f"edge Chords: {signed_chords}")
        print()


    vortexToRemove = list(verticesFromEdges(edgeSet))  # consume generator once
    print(f"vertices to remove are: {vortexToRemove}")

    matchingGraph = iso_vertecies(graph, vortexToRemove)
    print(f"matching graph:\n{matchingGraph.adjMatrix}")
    print(f"matching vertices are: {matchingGraph.vertexSet}")

    pairPoints = all_matchings(matchingGraph)
    pairpointsList=list(pairPoints)
    matchingV=matchingGraph.vertexSet
    
    print(f"pairs and points are: {pairpointsList}")
    print(f"vertecies to remove: {matchingV}")
    print(f"oriented cycles : {orientedCycles}")
    
    GS_results = CumulantGenerator(pairpointsList,matchingV, orientedCycles)
    for idx, GS in enumerate(GS_results):
        print(f"GS{idx} =", GS)

print("\n--- Processing no cycles  ---")

c0=[]
v0=graph.vertexSet
pp0=all_matchings(graph)
ppl0=list(pp0)
print(f"pairs and points are: {ppl0}")
GS_0 = CumulantGenerator(ppl0,v0, c0)
for idx, GS in enumerate(GS_0):
    print(f"GS={idx} =", GS)








Enter the size of the transition matrix (n x n):  4


Enter the transition matrix row by row (space-separated entries for each row of size 4):


Row 1:  0 1 1 0
Row 2:  1 0 1 1
Row 3:  1 1 0 1
Row 4:  0 1 1 0


{(0, 1), (1, 2), (2, 1), (3, 1), (2, 0), (2, 3), (0, 2), (1, 0), (3, 2), (1, 3)}
<class 'set'>

Chords: {(2, 3), (1, 2)}

Fundamental cycle 1: {(0, 1), (0, 2), (2, 3), (1, 3)}
Fundamental cycle 2: {(0, 1), (0, 2), (1, 2)}
Disjoint cycle 1 chords (2, 3): [(0, 1), (0, 2), (1, 3), (2, 3)]
Disjoint cycle 2 chords (1, 2): [(0, 1), (0, 2), (1, 2)]
Disjoint cycle 3 chords (1, 2) (2, 3): [(1, 2), (1, 3), (2, 3)]
disjoint cycle list [({(2, 3)}, {(0, 1), (0, 2), (2, 3), (1, 3)}), ({(1, 2)}, {(0, 1), (0, 2), (1, 2)}), ({(2, 3), (1, 2)}, {(2, 3), (1, 2), (1, 3)})]

All Even Degree Subgraphs:
  Even degree subgraph 1: [(0, 1), (0, 2), (1, 3), (2, 3)]
  Even degree subgraph 2: [(0, 1), (0, 2), (1, 2)]
  Even degree subgraph 3: [(1, 2), (1, 3), (2, 3)]
neighbors of 0 = [1, 2]
neighbors of 1 = [0, 2, 3]
neighbors of 2 = [0, 1, 3]
neighbors of 3 = [1, 2]
Disjoint cycle 1 chords (2, 3): [(0, 1), (0, 2), (1, 3), (2, 3)]
Disjoint cycle 2 chords (1, 2): [(0, 1), (0, 2), (1, 2)]
Disjoint cycle 3 chords (1, 

In [ ]:
0 1 1 0 0 0  
1 0 1 1 0 0  
1 1 0 0 0 0  
0 1 0 0 1 1
0 0 0 1 0 1
0 0 0 1 1 0

0 0 0 0 1 1 0 1 1 0 0 0
0 0 0 0 0 0 1 1 0 1 1 0
0 0 0 0 0 1 0 1 0 1 0 1
0 0 0 0 1 0 1 0 1 0 1 0
1 0 0 1 0 0 0 0 1 1 0 0
1 0 1 0 0 0 0 0 0 1 1 0
0 1 0 1 0 0 0 0 1 0 1 1
0 1 1 0 0 0 0 0 0 0 1 1
1 0 0 1 1 0 1 0 0 0 0 0
0 1 1 0 1 1 0 1 0 0 0 0
0 1 0 1 0 1 1 0 0 0 0 0
0 0 1 0 0 0 1 1 0 0 0 0


Row 1:  0 1 1 0
Row 2:  1 0 1 1
Row 3:  1 1 0 1
Row 4:  0 1 1 0


In [21]:
import itertools
import math

def filter_pairPoints(pairPoints, vertices_to_remove_copy):
    filtered = []
    for sublist in pairPoints:
        # Remove the whole sublist if it contains a tuple with any vertex to remove
        if any(isinstance(item, tuple) and any(v in vertices_to_remove_copy for v in item) for item in sublist):
            continue
        # Otherwise, remove only individual vertices
        new_sublist = [item for item in sublist if not (isinstance(item, int) and item in vertices_to_remove_copy)]
        if new_sublist:
            filtered.append(new_sublist)
    return filtered

def process_pairPoints(pairPoints_copy):
    expressions = []
    for group in pairPoints_copy:
        bracket_count = sum(1 for item in group if isinstance(item, tuple))
        sign = "-" if bracket_count % 2 == 1 else ""
        terms = []

        # First add all integers
        for item in group:
            if isinstance(item, int):
                terms.append(f"w{item}")
        # Then add all tuples as wij*wji
        for item in group:
            if isinstance(item, tuple):
                i, j = item
                terms.append(f"w{i}{j}*w{j}{i}")

        expr = f"{sign}{'*'.join(terms)}"
        expressions.append(f"({expr})")
    return "+".join(expressions)

def process_orientedCycles(orientedCycles):
    sign = 1
    sqrt_terms = []
    for cycle in orientedCycles:
        length = len(cycle)
        if length % 2 == 0:
            sign *= -1  # even length → flip sign
        else:
            sign *= 1   # odd length → keep sign

        # Collect forward and reverse products
        for i in range(length):
            i1 = cycle[i]
            i2 = cycle[(i + 1) % length]
            sqrt_terms.append(f"w{i1}{i2}")
        for i in range(length - 1, -1, -1):
            i1 = cycle[i]
            i2 = cycle[i - 1 if i > 0 else -1]
            sqrt_terms.append(f"w{i1}{i2}")

    sign_str = "+1" if sign == 1 else "-1"
    sqrt_expr = f"{sign_str}*2*sqrt(" + "*".join(sqrt_terms) + ")"
    return sqrt_expr

def CumulantGenerator(pairPoints, vertices_to_remove, orientedCycles):
    result = []

    for i in range(len(vertices_to_remove) + 1):
        combos = list(itertools.combinations(vertices_to_remove, i))
        gs_brackets = []

        for combo in combos:
            pairPoints_copy = filter_pairPoints(pairPoints, combo)
            gs_inner = process_pairPoints(pairPoints_copy)
            gs_brackets.append(f"[{gs_inner}]")

        # Combine all bracket expressions for this i
        GS_expression = "+".join(gs_brackets)
        sqrt_term = process_orientedCycles(orientedCycles)
        GS_full = f"{GS_expression}*{sqrt_term}"
        result.append(GS_full)

    return result



pairPoints = [
    [5, 4, 3],
    [(4, 5), 3],
    [5, (3, 4)],
    [4, (3, 5)]
]

vertices_to_remove = [3, 4, 5]
orientedCycles = [[0, 1, 2]]

results = CumulantGenerator(pairPoints, vertices_to_remove, orientedCycles)

for i, gs in enumerate(results):
    print(f"GS{i} = {gs}")



GS0 = [(w5*w4*w3)+(-w3*w45*w54)+(-w5*w34*w43)+(-w4*w35*w53)]*+1*2*sqrt(w01*w12*w20*w21*w10*w02)
GS1 = [(w5*w4)+(-w45*w54)]+[(w5*w3)+(-w35*w53)]+[(w4*w3)+(-w34*w43)]*+1*2*sqrt(w01*w12*w20*w21*w10*w02)
GS2 = [(w5)]+[(w4)]+[(w3)]*+1*2*sqrt(w01*w12*w20*w21*w10*w02)
GS3 = []*+1*2*sqrt(w01*w12*w20*w21*w10*w02)


In [ ]:
pairs and points are: [[5, 4, 3], [(4, 5), 3], [5, (3, 4)], [4, (3, 5)]]
vertecies to remove: [3, 4, 5]
oriented cycles : [[0, 1, 2]]

In [ ]:
-2f, 0, 0, 0, 0, 0, 0, p*exp(q_{y}), p*exp(q_{y}), 0, 0                 

0, -2f, 0, 0, 0, 0, 0, 0, 0, p*exp(-q_{y}), p*exp(-q_{y})

0, 0, -2f, 0, 0, p*exp(-q_{x}), 0, p*exp(-q_{x}), 0, 0, 0

0, 0, 0, -2f, p*exp(-q_{x}), 0, p*exp(-q_{x}), 0, 0, 0, 0

f*exp(-q_{x}), 0, 0, 0, -2f-p, 0, p, 0, f*exp(-q_{x}), f*exp(-q_{x}), 0

f*exp(q_{x}), 0, 0, 0, 0, -2f-p, 0, 0, f*exp(q_{x}), f*exp(q_{x}), 0

0, f*exp(-q_{x}), 0, 0, 0, 0, -2f-p, 0, 0, 0, f*exp(-q_{x})

0, f*exp(q_{x}), 0, 0, 0, 0, 0, -2f-p, 0, 0, f*exp(q_{x})

0, 0, 0, f*exp(q_{y}), f*exp(q_{y}), 0, f*exp(q_{y}), 0, -2f-p, 0, 0

0, 0, f*exp(q_{y}), 0, 0, f*exp(q_{y}), 0, f*exp(q_{y}), -2f-p, 0, 0

0, 0, 0, f*exp(-q_{y}), f*exp(-q_{y}), 0, 0, 0, 0, -2f-p, 0

0, 0, f*exp(-q_{y}), 0, 0, f*exp(-q_{y}), 0, 0, 0, 0, -2f-p